#program to remove the stop words from text and convert the file into a excel format
#applying TfIDF on the words , 
#Applying SVD on the bag of words.
#Code applicable for each podcast.
#Example code applied on a single podcast.

In [70]:
import shutil
import os
shutil.copytree('C:\\Users\\Harini\\Downloads\\Podknow-master\\data\\transcripts\\gcsst\\raw','C:\\Users\\Harini\\Final_Transcripts' )

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Harini\\Final_Transcripts'

In [71]:
os.getcwd()


'C:\\Users\\Harini'

#using nltk to remove stop words  and form a excel sheet.

In [72]:
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

In [159]:
results = []
import pandas as pd
for folderName,subfolders,fileName in os.walk('C:\\Users\\Harini\\Final_Transcripts'):
       for file in fileName:
            if  file.endswith(".txt"):
                scrubbed = []
                # file in the format of ID_Name_Date_TopicName.txt
                value0, value1, value2,value3,*extraWords = file.split('_')
                file = open(os.path.join(folderName,file),'rt')
                text = file.read() 
                tokens = word_tokenize(text)
                tokens = [w.lower() for w in tokens]
                table = str.maketrans('', '', string.punctuation)
                stripped = [w.translate(table) for w in tokens]
                words = [word for word in stripped if word.isalpha()]
                stop_words = set(stopwords.words('english'))
                #removing the stop words 
                words = [w for w in words if not w in stop_words]
                scrubbed.append(words)
                value4 = words
                #value5 needs to be changed based on the source
                #value5 = 'gcsst'
                rows = (value0,value1,value2,value3,value4)
                results.append(rows)            

labels = ['ID','Name','Date','topicName','scrubbedtext']
PodKnow_Data = pd.DataFrame.from_records(results, columns= labels)
print(PodKnow_Data)
PodKnow_Data.to_csv(r'PodKnow_Data.csv')

             ID                                               Name  \
0    1012570406                                    TheMFCEOProject   
1    1028908750                                        HiddenBrain   
2    1039708229                                      BrownAmbition   
3    1042368254                            TheHerdwithColinCowherd   
4    1043699613  Saints-Rams, Succession's Brilliance, Team USA...   
5    1044196249                                   MyDadWroteAPorno   
6    1047335260                                  TheBenShapiroShow   
7    1051557000                                          ScienceVs   
8    1057255460                              TheNPRPoliticsPodcast   
9    1062418176                                    TRUECRIMEGARAGE   
10   1068563276                  GettingCuriouswithJonathanVanNess   
11   1070322219                                       JockoPodcast   
12   1081124929                                 TheVanishedPodcast   
13   1087110764     

In [205]:
#Using the tfidfVectorizer to get the bag of words for every word in the podcast
#bag of words has all the words with the count 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df =1, stop_words= 'english')
#Considering podcast:-PodKnow_Data['scrubbedtext'][0]
bag_of_words = vectorizer.fit_transform(PodKnow_Data['scrubbedtext'][0])

In [230]:
PodKnow_Data['scrubbedtext'][0]

['stack',
 'hundreds',
 'roof',
 'stop',
 'reach',
 'family',
 'would',
 'nt',
 'food',
 'anybody',
 'guys',
 'listen',
 'mfceo',
 'project',
 'andy',
 'host',
 'motherfuking',
 'ceo',
 'guys',
 'always',
 'always',
 'say',
 'always',
 'join',
 'amazing',
 'handsome',
 'buff',
 'podcast',
 'host',
 'guys',
 'million',
 'nicknamesfor',
 'nicknames',
 'always',
 'nicknames',
 'dj',
 'dude',
 'dude',
 'hard',
 'de',
 'leon',
 'going',
 'believe',
 'well',
 'going',
 'actually',
 'really',
 'well',
 'one',
 'way',
 'key',
 'thing',
 'hard',
 'either',
 'going',
 'ask',
 'little',
 'questions',
 'drive',
 'crazy',
 'like',
 'actually',
 'question',
 'asking',
 'know',
 'moderate',
 'response',
 'good',
 'keeping',
 'keeping',
 'diet',
 'right',
 'problem',
 'way',
 'manifest',
 'dress',
 'lose',
 'appetite',
 'nt',
 'eat',
 'enough',
 'yeah',
 'disqualified',
 'yeah',
 'bro',
 'listen',
 'know',
 'saying',
 'like',
 'heart',
 'called',
 'hardgainer',
 'man',
 'yeah',
 'naturally',
 'thin',


In [206]:
bag_of_words.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [212]:
#applying Truncated SVD for the bag of words created. limiting it to 5 in thsi case
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
#consdiering 5 topics , this value can be changed based on coherence score vs number of topics.
svd = TruncatedSVD(n_components = 5)
lsa = svd.fit_transform(bag_of_words)

In [215]:
print(lsa.shape)

(2753, 5)


In [214]:
#For display of the topics
import pandas as pd

latent_topic_df = pd.DataFrame(lsa,columns = ["topic_1","topic_2","topic_3","topic_4","topic_5"])
latent_topic_df["body"] = PodKnow_Data['scrubbedtext'][0]
display(latent_topic_df[["body","topic_1","topic_2","topic_3","topic_4","topic_5"]])

,body,topic_1,topic_2,topic_3,topic_4,topic_5
0,stack,1.747688e-11,-1.769820e-10,-7.899669e-10,-1.916988e-10,1.324528e-08
1,hundreds,-1.834956e-11,1.619779e-10,2.317276e-10,3.366521e-10,-5.310464e-09
2,roof,-3.553287e-12,2.811195e-10,9.918316e-11,-5.397719e-10,-3.494424e-09
3,stop,-4.123283e-10,-3.287125e-10,-1.662509e-08,-6.369201e-09,-1.070919e-08
4,reach,-6.570897e-12,8.679574e-11,-1.469994e-10,1.687196e-09,5.315331e-10
5,family,-1.162814e-08,5.508592e-08,1.248344e-07,-2.868914e-08,6.535495e-07
6,would,9.827487e-21,-3.254022e-18,2.386984e-17,3.981343e-17,-1.704298e-16
7,nt,-3.639522e-07,3.076468e-06,9.999952e-01,8.839618e-06,-7.242065e-05
8,food,-1.178239e-09,7.454180e-09,2.775440e-08,3.771291e-08,-7.110343e-08
9,anybody,8.121465e-09,-4.838446e-08,-4.898981e-08,1.911299e-08,-5.798615e-07


In [223]:
#COde for VIsualization of lsa using bokeh
#import pandas as pd
#from bokeh.io import push_notebook, show, output_notebook
#from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, LabelSet
#output_notebook()

#df = pd.DataFrame(columns=['x', 'y', 'document'])
#df['x'], df['y'], df['document'] = lsa[:,1], lsa[:,4], range(len(PodKnow_Data['scrubbedtext'][0]))
 
#source = ColumnDataSource(ColumnDataSource.from_df(df))
#labels = LabelSet(x="x", y="y", text="document", y_offset=8,
#                  text_font_size="8pt", text_color="#555555",
#                  source=source, text_align='center')
 
#plot = figure(plot_width=600, plot_height=600)
#plot.circle("x", "y", size=5, source=source, line_color="black", fill_alpha=0.8)
#plot.add_layout(labels)
#show(plot, notebook_handle=True)
 

In [227]:
#Dictonary is the by product when we apply lsa, itcontains all the words used in the dcoument
dictionary = vectorizer.get_feature_names()
dictionary[:]

['ability',
 'able',
 'absol',
 'absolute',
 'absolutely',
 'accelerate',
 'action',
 'actually',
 'adam',
 'adams',
 'add',
 'adjustment',
 'advance',
 'advice',
 'age',
 'ago',
 'agree',
 'ai',
 'alabama',
 'alarm',
 'alison',
 'allison',
 'alright',
 'alternative',
 'amazing',
 'ambition',
 'amounts',
 'andy',
 'anger',
 'announced',
 'announcer',
 'annual',
 'answer',
 'answered',
 'anybody',
 'apparently',
 'appetite',
 'apply',
 'appreciate',
 'approval',
 'area',
 'arte',
 'ask',
 'asked',
 'asking',
 'aspect',
 'ass',
 'assume',
 'ate',
 'attention',
 'aunt',
 'authority',
 'away',
 'awesome',
 'awhile',
 'bad',
 'ball',
 'banners',
 'barely',
 'basically',
 'basics',
 'bdubs',
 'bed',
 'beer',
 'beginning',
 'believe',
 'ben',
 'best',
 'bet',
 'better',
 'big',
 'biggest',
 'bit',
 'black',
 'bloody',
 'body',
 'bodybuilding',
 'boost',
 'boys',
 'brain',
 'brains',
 'bright',
 'bro',
 'brother',
 'bucks',
 'buddies',
 'budweiser',
 'buff',
 'build',
 'bullshit',
 'bummer',
 

In [228]:
encoding_matrix = pd.DataFrame(svd.components_, index=['topic_1','topic_2','topic_3','topic_4','topic_5'],columns=dictionary).T

In [229]:
encoding_matrix
#Need to visualize this and find the similairity with respect to topic

,topic_1,topic_2,topic_3,topic_4,topic_5
ability,1.243416e-13,-1.518174e-12,1.241346e-11,2.200204e-11,-2.604697e-10
able,1.595953e-11,-7.630669e-11,-1.205694e-09,-9.319871e-11,2.469838e-08
absol,-3.595798e-13,2.944377e-12,1.065983e-11,-1.375294e-11,-1.472498e-10
absolute,-3.237778e-13,2.923868e-12,1.251907e-12,-4.112890e-11,1.621076e-11
absolutely,-2.116353e-11,1.104026e-09,-5.870052e-09,-1.052305e-08,-7.288776e-08
accelerate,-1.376854e-11,1.091192e-10,8.484155e-10,1.004842e-09,-1.632631e-08
action,-3.933208e-13,4.617772e-12,-7.819350e-13,-4.146790e-11,-6.121465e-12
actually,5.933091e-07,-1.038596e-05,-8.193675e-06,-7.812186e-05,1.125539e-03
adam,6.223206e-13,-2.193318e-12,-4.739001e-12,2.835167e-11,-7.183664e-11
adams,5.077071e-14,-5.953607e-13,4.101882e-15,7.599352e-12,-9.112042e-11


In [190]:
#This helps in sorting the values depending on the absolute value 
import numpy as np

encoding_matrix['abs_1'] = np.abs(encoding_matrix['topic_1'])
encoding_matrix['abs_2'] = np.abs(encoding_matrix['topic_2'])
encoding_matrix['abs_3'] = np.abs(encoding_matrix['topic_3'])
encoding_matrix['abs_4'] = np.abs(encoding_matrix['topic_4'])
encoding_matrix['abs_5'] = np.abs(encoding_matrix['topic_5'])
encoding_matrix.sort_values('abs_5',ascending=False)

,topic_1,topic_2,topic_3,topic_4,topic_5,abs_1,abs_2,abs_3,abs_4,abs_5
think,3.188458e-06,-2.557698e-05,4.288686e-06,2.150912e-04,9.999332e-01,3.188458e-06,2.557698e-05,4.288686e-06,2.150912e-04,9.999332e-01
theory,-1.224343e-04,1.632106e-03,-1.644734e-04,-2.335164e-03,6.215665e-03,1.224343e-04,1.632106e-03,1.644734e-04,2.335164e-03,6.215665e-03
terror,1.370276e-04,-2.684195e-03,-7.773549e-05,2.353421e-03,-6.137102e-03,1.370276e-04,2.684195e-03,7.773549e-05,2.353421e-03,6.137102e-03
going,2.386276e-06,1.175341e-03,1.007396e-03,5.729012e-04,-2.643391e-03,2.386276e-06,1.175341e-03,1.007396e-03,5.729012e-04,2.643391e-03
better,9.155371e-05,-8.214691e-04,3.234699e-04,-3.901319e-04,2.552179e-03,9.155371e-05,8.214691e-04,3.234699e-04,3.901319e-04,2.552179e-03
anxiety,-6.629701e-06,1.428365e-03,5.244942e-04,4.272047e-04,-2.377012e-03,6.629701e-06,1.428365e-03,5.244942e-04,4.272047e-04,2.377012e-03
trump,-2.964195e-05,2.784492e-05,2.105748e-04,-1.159718e-03,2.369540e-03,2.964195e-05,2.784492e-05,2.105748e-04,1.159718e-03,2.369540e-03
study,1.830801e-05,1.959226e-04,6.806470e-05,4.247767e-04,2.048814e-03,1.830801e-05,1.959226e-04,6.806470e-05,4.247767e-04,2.048814e-03
said,6.720953e-06,3.768313e-04,2.256426e-04,4.163165e-04,-1.838548e-03,6.720953e-06,3.768313e-04,2.256426e-04,4.163165e-04,1.838548e-03
brain,2.174942e-05,-1.095337e-03,8.828293e-05,-2.046017e-03,1.659738e-03,2.174942e-05,1.095337e-03,8.828293e-05,2.046017e-03,1.659738e-03


In [ ]:
#Visualization in progress